In [1]:
import numpy as np
import glob
import os
import pandas as pd
import scipy.signal as signal
import mne
import torch
from torch.utils.data import DataLoader, Dataset
from mne import preprocessing, Epochs
import utils
import matplotlib.pyplot as plt

## Data pre-processing

In [2]:
#create dataframe from CSV files
eeg_file_path = 'data/eeg_data_A/'
eog_file_path = 'data/eeg_data_B/'
#eeg_dataframe_A.head()
#eeg_dataframe_A = utils.format_df(eeg_dataframe_A)
#eeg_dataframe_A.head()
#eeg_dataframe = utils.gdf_to_df(file_path + 'A01T.gdf')


### Band-pass filtering (noise reduction)

In [3]:
eeg_files = glob.glob(os.path.join(eeg_file_path, '*.gdf'))
eog_files = glob.glob(os.path.join(eog_file_path + '*.gdf'))
eeg_channels = 25
eog_channels = 6
method = 'fastica'
raw_eeg_objects = []  # list to store raw objects
raw_eog_objects = []

eog_epoch_objects = []

rename_dict = {
    'EOG:ch01': 'EOG001',
    'EOG:ch02': 'EOG002',
    'EOG:ch03': 'EOG003'
}


for file in eeg_files + eog_files:
    raw = mne.io.read_raw_gdf(file, preload=True)
    if file in eog_files:
        # Rename EOG channels and set channel types
        raw.rename_channels(rename_dict)
        raw.set_channel_types({'EOG001': 'eog', 'EOG002': 'eog', 'EOG003': 'eog'})

        # Create EOG epochs
        eog_epochs = mne.preprocessing.create_eog_epochs(raw, baseline=(-0.5, -0.2))
        eog_epoch_objects.append(eog_epochs)
        raw_eog_objects.append(raw)
    else:
        raw_eeg_objects.append(raw)
    raw.filter(l_freq=1.0, h_freq=40.0, verbose=False)



Extracting EDF parameters from /run/media/thomas/hdd/repos/deep-learning/midterm_project/data/eeg_data_A/A09E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 675097  =      0.000 ...  2700.388 secs...


/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Extracting EDF parameters from /run/media/thomas/hdd/repos/deep-learning/midterm_project/data/eeg_data_A/A06E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 666372  =      0.000 ...  2665.488 secs...


/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Extracting EDF parameters from /run/media/thomas/hdd/repos/deep-learning/midterm_project/data/eeg_data_A/A09T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 673327  =      0.000 ...  2693.308 secs...


...

Using EOG channels: EOG001, EOG002, EOG003
EOG channel index for this subject is: [3 4 5]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG002 for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

Now detecting blinks and generating corresponding events
Found 33 significant peaks
Number of EOG events detected: 33
Not setting metadata
33 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 33 events and 251 original time points ...
0 bad epochs drop

/tmp/ipykernel_18568/13074463.py:19: RuntimeWarning: Highpass cutoff frequency 100.0 is greater than lowpass cutoff frequency 0.5, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_gdf(file, preload=True)


Using EOG channels: EOG001, EOG002, EOG003
EOG channel index for this subject is: [3 4 5]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG002 for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

Now detecting blinks and generating corresponding events
Found 101 significant peaks
Number of EOG events detected: 101
Not setting metadata
101 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 101 events and 251 original time points ...
0 bad epochs 

/tmp/ipykernel_18568/13074463.py:19: RuntimeWarning: Highpass cutoff frequency 100.0 is greater than lowpass cutoff frequency 0.5, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_gdf(file, preload=True)


Using EOG channels: EOG001, EOG002, EOG003
EOG channel index for this subject is: [3 4 5]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Selecting channel EOG002 for blink detection
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

Now detecting blinks and generating corresponding events
Found 32 significant peaks
Number of EOG events detected: 32
Not setting metadata
32 matching events found
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 32 events and 251 original time points ...
1 bad epochs drop

Convert to PyTorch Tensors

In [5]:
eeg_arrays = []
eog_arrays = []

for raw in raw_eeg_objects:
    data = raw.get_data()
    eeg_arrays.append(data)

for raw in raw_eog_objects:
    data = raw.get_data()
    eog_arrays.append(data)

eeg_arrays = utils.pad_arrays(eeg_arrays)
eog_arrays = utils.pad_arrays(eog_arrays)

tensor_eeg = torch.tensor(eeg_arrays,dtype=torch.float32)
tensor_eog = torch.tensor(eog_arrays.dtype=torch.float32)

print(tensor_eeg.shape)
print(tensor_eog.shape)

torch.Size([18, 25, 687792])
torch.Size([40, 6, 787729])
